In [32]:
reset()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  (), please run `%reset?` for details


In [1]:
import time
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import math
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression, Perceptron
import ipympl
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, f1_score, precision_recall_curve, average_precision_score, roc_curve, auc
%matplotlib widget
import warnings
warnings.filterwarnings('ignore')

### Carreguem el dataset

In [2]:
def load_dataset(path):
    d_train = pd.read_csv(path, header=0, delimiter=',')
    return d_train

os.getcwd()
path = 'C:/Users/julia/Documents/JULIA/UNI/3r/kaggle/train_gr'
os.chdir(path)
os.getcwd()
d_train = load_dataset('train.csv')

In [3]:
d_train.insert(3, 'words', d_train["user_review"].apply(lambda s: len(s.split())))
d_train = d_train.drop(['review_id'], axis=1)
d_train['title'] = [s.replace(" ", "") for s in d_train['title']]
d_train.insert(2, 'year_s', [str(s) for s in d_train['year']])
d_train.insert(0, 'title_review', d_train['title'] + " " + d_train['year_s'] + " " + d_train["user_review"])

### Separem els atributs de la X i l'atribut objectiu y

In [8]:
data = d_train.values
l = len(d_train.columns)
X = data[:, 0]
y = data[:, l-1]
y = y.astype('int')

#### Funció per calcular el temps i la precisió d'un model

In [10]:
def temps_score_model(clf, x_t, x_v, y_t, y_v):
    inici = time.time()
    clf = clf.fit(x_t, y_t)
    fi = time.time()
    print("Temps: " + str(fi-inici))
    print("Score: " + str(clf.score(x_v, y_v)))

#### Separem les dades en test i validation

In [11]:
x_t, x_v, y_t, y_v = train_test_split(X, y, train_size=0.8, stratify=y)
vectorizer = CountVectorizer()  

### Provem diferents models

In [12]:
clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=12.0, fit_intercept=True, penalty='l2', tol=0.1))])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 1.8811931610107422
Score: 0.8656759074021149


In [13]:
vectorizer = CountVectorizer(stop_words='english')  
clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier(random_state=0, max_depth=2))])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 1.6740975379943848
Score: 0.6418976850528723


In [14]:
vectorizer = CountVectorizer(stop_words='english')  
clf = Pipeline([('vect', vectorizer),('tfidf', TfidfTransformer()),
                ('clf', Perceptron(tol=0.005, random_state=0))])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 1.5243017673492432
Score: 0.8248070877393541


In [15]:
vectorizer = CountVectorizer(stop_words='english')  
clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier(n_neighbors=3))])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 1.4180030822753906
Score: 0.7627893683909689


In [16]:
vectorizer = CountVectorizer(stop_words='english')  
clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', BaggingClassifier(KNeighborsClassifier(n_neighbors=3), max_samples=0.5, max_features=0.5))])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 1.9787817001342773
Score: 0.5973135181480423


In [17]:
vectorizer = CountVectorizer(stop_words='english')  
clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', AdaBoostClassifier(n_estimators=100))])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 12.230507135391235
Score: 0.8042297799371249


In [18]:
vectorizer = CountVectorizer(stop_words='english')  
clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB(alpha=1.0))])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 1.3908004760742188
Score: 0.7999428408116604


In [19]:
vectorizer = CountVectorizer(stop_words='english')  
clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', ComplementNB())])
temps_score_model(clf, x_t, x_v, y_t, y_v)

Temps: 1.4862914085388184
Score: 0.8276650471563304


### Pels models que han tingut millor score trobem els millors hiperparàmetres

#### Logistic

In [20]:
maxi = 0
l = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
for c in l:#range(1,20):
    clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=c, fit_intercept=True, penalty='l2', tol=0.1))])
    clf = clf.fit(x_t, y_t)
    m = clf.score(x_v, y_v)
    if (m > maxi):
        maxi = m
        c_max = c
print("Millor c:    ", c_max)
print("Score màxim: ", maxi)

Millor c:     3.5
Score màxim:  0.8628179479851386


In [21]:
folds = 5
maxi = 0
l = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]
for t in l:
    clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=3.5, fit_intercept=True, penalty='l2', tol=t))])
    clf = clf.fit(x_t, y_t)
    m = clf.score(x_v, y_v)
    if (m > maxi):
        maxi = m
        t_max = t
print("Millor t:    ", t_max)
print("Score màxim: ", maxi)

Millor t:     0.1
Score màxim:  0.8628179479851386


#### ComplementNB

In [22]:
maxi = 0
l = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
for c in range(1,20):
    clf = Pipeline([('vect', vectorizer), ('tfidf', TfidfTransformer()),
                ('clf', ComplementNB(alpha=c))])
    clf = clf.fit(x_t, y_t)
    m = clf.score(x_v, y_v)
    if (m > maxi):
        maxi = m
        c_max = c
print("Millor alpha: ", c_max)
print("Score màxim:  ", maxi)

Millor alpha:  1
Score màxim:   0.8276650471563304


#### Perceptron

In [23]:
maxi = 0
l = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]
for c in l:
    clf = Pipeline([('vect', vectorizer), 
                   ('tfidf', TfidfTransformer()),
                   ('clf', Perceptron(tol=c, random_state=0)),
                   ])
    clf = clf.fit(x_t, y_t)
    m = clf.score(x_v, y_v)
    if (m > maxi):
        maxi = m
        c_max = c
print("Millor tol:  ", c_max)
print("Score màxim: ", maxi)

Millor tol:   0.01
Score màxim:  0.827950843098028
